In [1]:
# Run once
#%pip install transformers accelerate datasets

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class ConfigDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = f"Input: {item['input']}\nOutput: {item['output']}"
        encoding = self.tokenizer(input_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze()
        }

In [3]:
# Your provided dataset
data = [
    {
        "input": "Configure the eth28 ethernet device with the static IPv4 address 232.162.200.174/25",
        "output": "---\ninterfaces:\n- name: eth28\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 232.162.200.174\n      prefix-length: 25"
    },
    {
        "input": "Set the eth1 ethernet device with the IPv4 address 192.168.1.1/24",
        "output": "---\ninterfaces:\n- name: eth1\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 192.168.1.1\n      prefix-length: 24"
    },
    {
        "input": "Configure the eth28 ethernet device with the IPv4 address 232.162.200.174/25",
        "output": "---\ninterfaces:\n- name: eth28\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 232.162.200.174\n      prefix-length: 25"
    },
    {
        "input": "Set the eth1 ethernet device with the static IPv4 address 192.168.1.1/24",
        "output": "---\ninterfaces:\n- name: eth1\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 192.168.1.1\n      prefix-length: 24"
    },
    {
        "input": "Assign the eth0 ethernet device with the IPv4 address 10.0.0.1/8",
        "output": "---\ninterfaces:\n- name: eth0\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 10.0.0.1\n      prefix-length: 8"
    },
    {
        "input": "Configure the eth2 ethernet device with the IPv4 address 172.16.0.1/16",
        "output": "---\ninterfaces:\n- name: eth2\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 172.16.0.1\n      prefix-length: 16"
    },
    {
        "input": "Set the eth3 ethernet device with the IPv4 address 192.168.100.1/24",
        "output": "---\ninterfaces:\n- name: eth3\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 192.168.100.1\n      prefix-length: 24"
    },
    {
        "input": "Assign the eth4 ethernet device with the IPv4 address 10.10.10.1/24",
        "output": "---\ninterfaces:\n- name: eth4\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 10.10.10.1\n      prefix-length: 24"
    },
    {
        "input": "Configure the eth5 ethernet device with the static IPv4 address 172.31.255.255/12",
        "output": "---\ninterfaces:\n- name: eth5\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 172.31.255.255\n      prefix-length: 12"
    },
    {
        "input": "Set the eth6 ethernet device with the IPv4 address 192.0.2.1/24",
        "output": "---\ninterfaces:\n- name: eth6\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 192.0.2.1\n      prefix-length: 24"
    },
    {
        "input": "Assign the eth7 ethernet device with the IPv4 address 203.0.113.1/24",
        "output": "---\ninterfaces:\n- name: eth7\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 203.0.113.1\n      prefix-length: 24"
    },
    {
        "input": "Configure the eth8 ethernet device with the IPv4 address 198.51.100.1/24",
        "output": "---\ninterfaces:\n- name: eth8\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 198.51.100.1\n      prefix-length: 24"
    },
    {
        "input": "Set up the eth9 ethernet device with the IPv4 address 10.1.1.1/24",
        "output": "---\ninterfaces:\n- name: eth9\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 10.1.1.1\n      prefix-length: 24"
    },
    {
        "input": "Configure the eth10 ethernet device with the static IPv4 address 192.168.200.1/29",
        "output": "---\ninterfaces:\n- name: eth10\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 192.168.200.1\n      prefix-length: 29"
    },
    {
        "input": "Assign the eth11 ethernet device with the IPv4 address 172.20.0.1/20",
        "output": "---\ninterfaces:\n- name: eth11\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 172.20.0.1\n      prefix-length: 20"
    },
    {
        "input": "Set the eth12 ethernet device with the IPv4 address 10.5.5.1/16",
        "output": "---\ninterfaces:\n- name: eth12\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 10.5.5.1\n      prefix-length: 16"
    },
    {
        "input": "Configure the eth13 ethernet device with the IPv4 address 192.168.50.1/25",
        "output": "---\ninterfaces:\n- name: eth13\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 192.168.50.1\n      prefix-length: 25"
    },
    {
        "input": "Assign the eth14 ethernet device with the IPv4 address 172.31.10.1/24",
        "output": "---\ninterfaces:\n- name: eth14\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 172.31.10.1\n      prefix-length: 24"
    },
    {
        "input": "Set the eth15 ethernet device with the IPv4 address 10.255.255.1/8",
        "output": "---\ninterfaces:\n- name: eth15\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 10.255.255.1\n      prefix-length: 8"
    },
    {
        "input": "Configure the eth16 ethernet device with the IPv4 address 198.51.100.100/24",
        "output": "---\ninterfaces:\n- name: eth16\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 198.51.100.100\n      prefix-length: 24"
    },
    {
        "input": "Assign the eth17 ethernet device with the IPv4 address 192.168.1.100/24",
        "output": "---\ninterfaces:\n- name: eth17\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 192.168.1.100\n      prefix-length: 24"
    },
    {
        "input": "Set the eth18 ethernet device with the static IPv4 address 10.10.1.1/16",
        "output": "---\ninterfaces:\n- name: eth18\n  type: ethernet\n  state: up\n  ipv4:\n    enabled: true\n    dhcp: false\n    address:\n    - ip: 10.10.1.1\n      prefix-length: 16"
    },
]

In [4]:
# Split the data
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.pad_token_id = model.config.eos_token_id

# Create datasets and dataloaders
train_dataset = ConfigDataset(train_data, tokenizer)
val_dataset = ConfigDataset(val_data, tokenizer)

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Training settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 145  # Increased number of epochs
num_training_steps = num_epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Average train loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Average validation loss: {avg_val_loss:.4f}")

/opt/app-root/lib64/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/145: 100%|██████████| 3/3 [00:00<00:00,  3.62it/s]


Average train loss: 8.0626
Average validation loss: 9.3394


Epoch 2/145: 100%|██████████| 3/3 [00:00<00:00,  4.73it/s]


Average train loss: 7.9450
Average validation loss: 9.0176


Epoch 3/145: 100%|██████████| 3/3 [00:00<00:00,  4.75it/s]


...

Average train loss: 0.0553
Average validation loss: 0.0587


Epoch 144/145: 100%|██████████| 3/3 [00:00<00:00,  4.64it/s]


Average train loss: 0.0641
Average validation loss: 0.0586


Epoch 145/145: 100%|██████████| 3/3 [00:00<00:00,  4.65it/s]

Average train loss: 0.0524
Average validation loss: 0.0586


In [6]:
def generate_yaml(input_text):
    model.eval()
    input_text = f"Input: {input_text}\nOutput:"
    inputs = tokenizer.encode_plus(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    output = model.generate(
        input_ids, 
        attention_mask=attention_mask,
        max_length=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2, 
        top_k=50,
        top_p=0.95,
        temperature=0.01,
        do_sample=True,
        pad_token_id=model.config.eos_token_id
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    yaml_output = generated_text.split("Output:")[1].strip()
    return yaml_output

# Test examples
test_inputs = [
    "Configure the eth3 ethernet device with the static IPv4 address 232.162.200.174/25",
    "Set the eth1 ethernet device with the static IPv4 address 192.168.1.1/24",
    "Assign the eth2 ethernet device with the IPv4 address 10.0.0.1/8",
]

for test_input in test_inputs:
    print(f"Input: {test_input}")
    print(f"Generated YAML:\n{generate_yaml(test_input)}")
    print()

Input: Configure the eth3 ethernet device with the static IPv4 address 232.162.200.174/25
Generated YAML:
---
interfaces:
- name: eth7
  type: ether
 state: up
 ipv4: false
 dhcp: true
 address:

Input: Set the eth1 ethernet device with the static IPv4 address 192.168.1.1/24
Generated YAML:
---
interfaces:
- name: eth0
  type: ether
 state: up
 ipv4: false
 dhcp: true
 address: 192:168:1:24

Input: Assign the eth2 ethernet device with the IPv4 address 10.0.0.1/8
Generated YAML:
---
interfaces:
- name: eth1
  type: ether
 state: up
 ipv4: false
 dhcp: true
 address: 10
    enabled: True
 prefix-length: 8

